<a href="https://colab.research.google.com/github/nutapol97/Python-for-DS-AI_Nutapol_T./blob/main/Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Linear Regression

Bunch

In [632]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
boston = load_boston()

In [633]:
X = boston.data
y = boston.target

In [634]:
m=X.shape[0]    # No. samples
n=X.shape[1]    # No. features

**Standardization of datasets** is a common requirement for many machine learning estimators implemented in scikit-learn; they might behave badly if the individual features do not more or less look like standard normally distributed data: Gaussian with zero mean and unit variance.

In [635]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

Split Datas


In [636]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

Add intercepts

np.concatenate : Join a sequence of arrays along an existing axis.

In [637]:
intercept = np.ones((X_train.shape[0], 1))
X_train = np.concatenate((intercept, X_train), axis=1)
np.shape(X_train)

(354, 14)

In [638]:
intercept = np.ones((X_test.shape[0], 1))
X_test = np.concatenate((intercept, X_test), axis=1)
np.shape(X_test)

(152, 14)

Step 2: Fit your algorithm
1. Define your algorithm

In [641]:
from numpy.linalg import inv
def closed_form(X, y):
    return inv(X.T @ X) @ X.T @ y

In [642]:
theta = closed_form(X_train, y_train)
theta

array([22.5114854 , -1.02908523,  1.14529114,  0.7216558 ,  0.29838682,
       -2.06442942,  3.09418547, -0.0725698 , -3.14573302,  2.79193278,
       -2.6574623 , -1.97708707,  0.94441182, -3.72884154])

In [643]:
yhat = X_test @ theta
yhat

array([22.91225154, 23.71535879,  8.29744937, 24.65444106, 11.49053038,
        9.15722108, 28.58457474, 21.86559229, 27.29621119, 39.51561257,
       15.5023435 , 30.95271165, 22.98581825, 19.5120485 , 28.79894629,
       18.27401951, 25.23033572, 28.58586844, 23.03391477, 20.32206395,
       22.52863201, 30.04217991, 24.18476245, 10.67323183, 12.69197056,
       21.28182104,  8.22891551, 19.38528571, 27.59639137, 24.55706324,
       27.29300011, 15.91468939, 21.87681734, 13.54489741, 19.93272198,
       28.7963338 , 18.08719504, 22.54932974, 33.50372441, 18.25590237,
       13.93927185, 25.15862599, 14.05408303, 22.37432525, 18.19650791,
       29.13014493, 24.71234489, 30.08913994, 21.34382517, 30.68083575,
       18.00019991, 24.27254542, 24.1218011 , 29.05571932, 22.29187806,
       29.19003065, 13.95210308, 25.83761815, 19.57916217, 28.65299977,
       36.23738268,  4.95150792, 16.37936799, 21.55139064, 13.99717269,
       21.21148236, 18.31743552, 19.07497848, 34.45984538, 30.00

In [645]:
mse = ((y_test - yhat)**2).sum() / X_test.shape[0]

Algorithm 2: Batch Gradient Descent

In [715]:
from time import time

# Step 1: Prepare your data
# X_train, X_test have intercepts that are being concatenated to the data
# [1, features
#  1, features....]

# making sure our X_train has same sample size as y_train
assert X_train.shape[0] == y_train.shape[0]

# initialize our w
# We don't have to do X.shape[1] + 1 because our X_train already has the
# intercept
# w = theta/beta/coefficients
theta = np.zeros(X_train.shape[1])

# define the learning rate
# later on, you gonna know that it should be better to make it slowly decreasing
# once we perform a lot of iterations, we want the update to slow down, so it converges better
alpha = 0.0001

# define our max_iter
# typical to call it epochs <---ml people likes to call it
max_iter = 1000

loss_old = 10000

tol = 0.0001

iter_stop = 0

def h_theta(X, theta): # theta dot X
    return X @ theta

def mse(yhat, y):    # The loss function is the mean squared error
    return ((yhat - y)**2).sum() / yhat.shape[0]

def gradient(X, error):
    return X.T @ error

start = time()

# define your for loop
for i in range(max_iter):
    
    # 1. yhat = X @ w
    # prediction
    # yhat (m, ) = (m, n) @ (n, )
    yhat = h_theta(X_train, theta)

    # 2. error = yhat - y_train
    # error for use to calculate gradients
    # error (m, ) = (m, ) - (m, )
    error = yhat - y_train

    # 3. grad = X.T @ error
    # grad (n, ) = (n, m) @ (m, )
    # grad for each feature j
    grad = gradient(X_train, error)

    # 4. w = w - alpha * grad
    # update w
    # w (n, ) = (n, ) - scalar * (n, )
    theta = theta - alpha * grad  # update

time_taken = time() - start

2. Compute accuracy/loss


In [716]:
# we got our lovely w
# now it's time to check our accuracy
# 1. Make prediction
yhat = h_theta(X_test, theta)

# 2. Calculate mean squared errors
mse = mse(yhat, y_test)

# print the mse
print("MSE: ", mse)
print("Stop at iteration: ", iter_stop)
print("Time used: ", time_taken)

MSE:  23.136055274319066
Stop at iteration:  0
Time used:  0.009975194931030273



=== Task ===

1.Implement early stopping in which if the absolute difference between old loss and new loss does not exceed certain threshold, we abort the learning.



In [717]:
from time import time
import math
# making sure our X_train has same sample size as y_train
assert X_train.shape[0] == y_train.shape[0]
# Set theta at start point
theta = np.zeros(X_train.shape[1])
# Set Learning rate (alpha)
alpha = 0.0001
# Set Number of interlation that calculatetion
max_iter = 1000
# Set loss at start point for calculate. That should more big.
loss_old = 10000
# Ste tol for get the optmum point protect over fit
tol = 0.0001
iter_stop = 0

def h_theta(X, theta):
    return X @ theta

def mse(yhat, y):
    return ((yhat - y)**2).sum() / yhat.shape[0]

def gradient(X, error):
    return X.T @ error
start = time()
for i in range(max_iter):
    
    # 1. yhat = X @ w
    # prediction
    # yhat (m, ) = (m, n) @ (n, )
    #### X train dot theta(Learning rate)####
    yhat = h_theta(X_train, theta)
    loss_current = mse(yhat,y_train)
    
    different=np.abs(loss_old-loss_current)
    
    if different == tol or different < tol:
      break
    loss_old=loss_current
    # 2. error = yhat - y_train
    # error for use to calculate gradients
    # error (m, ) = (m, ) - (m, )
    error = yhat - y_train
    #print(error)
    # 3. grad = X.T @ error
    # grad (n, ) = (n, m) @ (m, )
    # grad for each feature j
    grad = gradient(X_train, error)

    # 4. w = w - alpha * grad
    # update w
    # w (n, ) = (n, ) - scalar * (n, )
    theta = theta - alpha * grad
    iter_stop +=1
time_taken = time() - start
# we got our lovely w
# now it's time to check our accuracy
# 1. Make prediction
yhat = h_theta(X_test, theta)

# 2. Calculate mean squared errors
mse = mse(yhat, y_test)

# print the mse
print("MSE: ", mse)
print("Stop at iteration: ", iter_stop)
print("Time used: ", time_taken)


MSE:  23.110826901477754
Stop at iteration:  851
Time used:  0.01662588119506836




1.    ***`assert`*** statement exists in almost every programming language. It helps detect problems early in your program, where the cause is clear, rather than later when some other operation fails.




2.Implement options for stochastic gradient descent in which we use only one sample for training. Make sure that sample does not repeat unless all samples are read at least once already.



Stochastic Gradient Descent

In [596]:
from time import time
import math,random
# making sure our X_train has same sample size as y_train
assert X_train.shape[0] == y_train.shape[0]

# Set theta at start point
theta = np.zeros(X_train.shape[1])
# Set Learning rate (alpha)
alpha = 0.0001
# Set Number of interlation that calculatetion
max_iter = 1000
# Set loss at start point for calculate. That should more big.
loss_old = 10000
# Ste tol for get the optmum point protect over fit
tol = 0.0001
iter_stop = 0

def h_theta(X, theta):
    return X @ theta

def mse(yhat, y):
    return ((yhat - y)**2).sum() / yhat.shape[0]

def gradient(X, error):
    return X.T @ error

def sto(X_train,y_train):
  no_random=random.randrange(X_train.shape[0])
  X_train=X_train[no_random,:]
  X_train=X_train.reshape(1,14)
  y_train=y_train[no_random:no_random+1]
 return X_train,y_train

#X_train,y_train=sto(X_train,y_train)
start = time()
for i in range(max_iter):
    
    # 1. yhat = X @ w
    # prediction
    # yhat (m, ) = (m, n) @ (n, )
    #### X train dot theta(Learning rate)####
    yhat = h_theta(X_train, theta)
    loss_current = mse(yhat,y_train)
    
    different=np.abs(loss_old-loss_current)
    
    if different == tol or different < tol:
      break
    loss_old=loss_current
    # 2. error = yhat - y_train
    # error for use to calculate gradients
    # error (m, ) = (m, ) - (m, )
    error = yhat - y_train
    #print(error)
    # 3. grad = X.T @ error
    # grad (n, ) = (n, m) @ (m, )
    # grad for each feature j
    grad = gradient(X_train, error)

    # 4. w = w - alpha * grad
    # update w
    # w (n, ) = (n, ) - scalar * (n, )
    theta = theta - alpha * grad
    iter_stop +=1
time_taken = time() - start
# we got our lovely w
# now it's time to check our accuracy
# 1. Make prediction
yhat = h_theta(X_test, theta)

# 2. Calculate mean squared errors
mse = mse(yhat, y_test)

# print the mse
print("MSE: ", mse)
print("Stop at iteration: ", iter_stop)
print("Time used: ", time_taken)


MSE:  30.180133878751096
Stop at iteration:  791
Time used:  0.014987468719482422


3 .Add options for mini-batch gradient descent.

Mini-Batch Gradient Descent

In [606]:
from time import time
import math,random
# making sure our X_train has same sample size as y_train
assert X_train.shape[0] == y_train.shape[0]

# Set theta at start point
theta = np.zeros(X_train.shape[1])
# Set Learning rate (alpha)
alpha = 0.0001
# Set Number of interlation that calculatetion
max_iter = 1000
# Set loss at start point for calculate. That should more big.
loss_old = 10000
# Ste tol for get the optmum point protect over fit
tol = 0.0001
iter_stop = 0

def h_theta(X, theta):
    return X @ theta

def mse(yhat, y):
    return ((yhat - y)**2).sum() / yhat.shape[0]

def gradient(X, error):
    return X.T @ error

def mini_batch(X_train,y_train,sizing):
  random_np= np.random.randint(X_train.shape[0],size=sizing)
  X_train=X_train[random_np,:]
  y_train=y_train[random_np]

  return X_train,y_train

X_train,y_train=mini_batch(X_train,y_train,300)
start = time()
for i in range(max_iter):
    
    # 1. yhat = X @ w
    # prediction
    # yhat (m, ) = (m, n) @ (n, )
    #### X train dot theta(Learning rate)####
    yhat = h_theta(X_train, theta)
    loss_current = mse(yhat,y_train)
    
    different=np.abs(loss_old-loss_current)
    
    if different == tol or different < tol:
      break
    loss_old=loss_current
    # 2. error = yhat - y_train
    # error for use to calculate gradients
    # error (m, ) = (m, ) - (m, )
    error = yhat - y_train
    #print(error)
    # 3. grad = X.T @ error
    # grad (n, ) = (n, m) @ (m, )
    # grad for each feature j
    grad = gradient(X_train, error)

    # 4. w = w - alpha * grad
    # update w
    # w (n, ) = (n, ) - scalar * (n, )
    theta = theta - alpha * grad
    iter_stop +=1
time_taken = time() - start
# we got our lovely w
# now it's time to check our accuracy
# 1. Make prediction
yhat = h_theta(X_test, theta)

# 2. Calculate mean squared errors
mse = mse(yhat, y_test)

# print the mse
print("MSE: ", mse)
print("Stop at iteration: ", iter_stop)
print("Time used: ", time_taken)


MSE:  37.7086932388522
Stop at iteration:  1000
Time used:  0.019178390502929688


Put everything into class.

In [731]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from time import time
import math
#assert X_train.shape[0] == y_train.shape[0]
boston = load_boston()
X = boston.data
y = boston.target
m=X.shape[0]    # No. samples
n=X.shape[1]    # No. features
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
intercept = np.ones((X_train.shape[0], 1))
X_train = np.concatenate((intercept, X_train), axis=1)
intercept = np.ones((X_test.shape[0], 1))
X_test = np.concatenate((intercept, X_test), axis=1)


class Linear_Regression:

  def __init__(self, alpha = 0.0001,max_iter = 1000,loss_old = 10000,
               tol = 0.0001,method='Batch'):
    self.alpha=alpha
    self.max_iter=max_iter
    self.loss_old= loss_old
    self.tol=tol
    self.method=method
    
  def h_theta(self,X,theta):
      return X @ theta

  def mse(self,yhat, y):
      return ((yhat - y)**2).sum() / yhat.shape[0]

  def gradient(self,X, error):
      return X.T @ error
  def sto(self,X_train,y_train):
      no_random=random.randrange(X_train.shape[0])
      X_train=X_train[no_random,:]
      X_train=X_train.reshape(1,14)
      y_train=y_train[no_random:no_random+1]
      return X_train,y_train
  def mini_batch(self,X_train,y_train,sizing):
      random_np= np.random.randint(X_train.shape[0],size=sizing)
      X_train=X_train[random_np,:]
      y_train=y_train[random_np]
      return X_train,y_train
  def fit(self,X_train,y_train):
      theta = np.zeros(X.shape[1])
      iter_stop=0
      loss_old=self.loss_old
      theta = np.zeros(X_train.shape[1])
      if self.method == 'sto':
        X_train,y_train=self.sto(X_train,y_train)
      if self.method == 'mini_batch':
        X_train,y_train=self.mini_batch(X_train,y_train,300)
      print(np.shape(X_train))
      for i in range(self.max_iter):
              
        # 1. yhat = X @ w
        # prediction
        # yhat (m, ) = (m, n) @ (n, )
        #### X train dot theta(Learning rate)####
        yhat = self.h_theta(X_train,theta)
        loss_current = self.mse(yhat,y_train)       
        different=np.abs(loss_old-loss_current)        
        if different == self.tol or different < self.tol:
            break
        loss_old=loss_current
        # 2. error = yhat - y_train
        # error for use to calculate gradients
        # error (m, ) = (m, ) - (m, )
        error = yhat - y_train
        #print(error)
        # 3. grad = X.T @ error
        # grad (n, ) = (n, m) @ (m, )
        # grad for each feature j
        grad = self.gradient(X_train, error)

        # 4. w = w - alpha * grad
        # update w
        # w (n, ) = (n, ) - scalar * (n, )
        theta = theta - self.alpha * grad
        iter_stop +=1
          
          
model = Linear_Regression(method ='sto')
model.fit(X_train, y_train)
yhat = model.h_theta(X_test,theta)
mse = model.mse(yhat, y_test)
print("MSE: ", mse)
print("iter_stop: ", iter_stop)

(1, 14)
MSE:  24.278351071417728
iter_stop:  851


In [732]:
print("MSE: ", mse)

MSE:  24.278351071417728
